In [2]:
import tensorflow as tf
import io

2023-03-30 20:18:22.830676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 20:18:22.928519: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 20:18:22.929685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 20:18:24.032881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
VOCAB_SIZE = 10000

words_file = io.open('./metadata.tsv', encoding='utf-8')
words = []
words_file.readline()
lines = words_file.readlines()
for line in lines:
    words.append(line.strip('\n '))

In [4]:
# 可以通过vocabulary的参数来初始化，需要注意的是，需要排除第一个'unk'字符
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=100,
    vocabulary=words)

2023-03-30 20:18:33.024989: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# 得出的结果与保存前一致
model = tf.keras.Sequential()
model.add(vectorize_layer)
model.predict(['I have lots of money'])

1/1 [==============================] - 0s 312ms/step


array([[ 10,  25, 736,   5, 275,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [6]:
vectorize_layer.vocabulary_size()

10000

In [7]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'was',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'on',
 'not',
 'you',
 'are',
 'his',
 'have',
 'be',
 'he',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'her',
 'just',
 'or',
 'about',
 'has',
 'out',
 'if',
 'some',
 'there',
 'what',
 'good',
 'more',
 'when',
 'very',
 'even',
 'she',
 'up',
 'my',
 'no',
 'would',
 'time',
 'only',
 'which',
 'really',
 'story',
 'their',
 'see',
 'were',
 'had',
 'can',
 'me',
 'we',
 'than',
 'much',
 'well',
 'been',
 'will',
 'get',
 'people',
 'bad',
 'also',
 'other',
 'do',
 'into',
 'great',
 'first',
 'because',
 'how',
 'most',
 'him',
 'dont',
 'made',
 'movies',
 'then',
 'them',
 'way',
 'films',
 'make',
 'could',
 'any',
 'after',
 'too',
 'characters',
 'think',
 'watch',
 'being',
 'two',
 'many',
 'seen',
 'character',
 'plot',
 'little',
 'never',
 'acting',
 'where',
 'best',
 '

In [9]:
# 导入embedding向量权重
emb_file = io.open('./vectors.tsv')
emb_lines = emb_file.readlines()
emb = []
for line in emb_lines:
    emb.append(line.split('\t'))
    
# 使用权重初始化Embedding层
embedding_layer = tf.keras.layers.Embedding(10000, 16, weights=[emb])
model = tf.keras.Sequential()
# 加入vectirze层
model.add(vectorize_layer)
model.add(embedding_layer)
model.compile('rmsprop','mse')
# 预测，输出(batch_size, length, embedding_dimension)
model.predict(['I have lots of money']).shape

1/1 [==============================] - 0s 242ms/step


(1, 100, 16)

In [10]:
weights = tf.constant([[1,0],[0,1]])
embedding_layer = tf.keras.layers.Embedding(2, 2,
                                            input_length=2,
                                           weights=[weights])
model = tf.keras.Sequential()
model.add(embedding_layer)
model.compile('rmsprop','mse')
import numpy as np
input_array = np.random.randint(2, size=(2, 2))
output_array = model.predict(input_array)
print(output_array)

1/1 [==============================] - 0s 68ms/step
[[[0. 1.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]]]
